In [1]:
import pandas as pd
import ast
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
from shutil import copyfile
import json
import random
import numpy as np
tqdm.pandas()

In [2]:
# Install W&B 
!pip install -q --upgrade wandb

# Login 
import wandb


! wandb login -

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
!mkdir -p /kaggle/working/datasets/coco/{images,labels}/{train,val}

In [4]:
vide_data = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")

In [5]:
width,height=1280,720
def parse_annotations(annotation): # [{'x': 559, 'y': 213, 'width': 50, 'height': 32}] -> [559, 213, 50, 32]
    changed_anno=''
    for ano in annotation:
        comp=list(ano.values())
        comp[0]=(comp[0]+np.round(comp[2]/2))/width#(comp[0]+comp[2])/(2*width)
        comp[1]=(comp[1]+np.round(comp[3]/2))/height#(comp[1]+comp[3])/(2*height)
        comp[2]/=width
        comp[3]/=height
        changed_anno+=f'0 {comp[0]} {comp[1]} {comp[2]} {comp[3]} \n'
    return changed_anno[:-1]
def make_path(image_info): #assuming a path starting from this files's location
    return "../input/tensorflow-great-barrier-reef/train_images/video_{0}/{1}.jpg".format(image_info['video_id'],image_info['video_frame'])

In [6]:
vide_data['annotations']=vide_data['annotations'].apply(lambda x: ast.literal_eval(x)) 

vide_data['num_bbox']=vide_data['annotations'].apply(lambda x: len(x))

keep=[num_bbox>0 or random.random()>0.96 for num_bbox in vide_data['num_bbox']]
vide_data=vide_data[keep]

vide_data['bboxes']=vide_data['annotations'].apply(parse_annotations)

vide_data["image_path"] = vide_data.apply(make_path,axis=1)

vide_data=vide_data.reset_index(drop=True)

In [7]:
#yetted this for kfold implementation
kf = GroupKFold(n_splits = 6)
vide_data['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(vide_data, y = vide_data.video_id.tolist(), groups=vide_data.sequence)):
    vide_data.loc[val_idx, 'fold'] = fold

In [8]:
SELECTED_FOLD = 4

In [9]:
trainf= open('/kaggle/working/datasets/coco/train.txt', "a")
valf= open('/kaggle/working/datasets/coco/val.txt', "a")

for i in tqdm(range(len(vide_data))):
    row = vide_data.loc[i]
    if row.fold != SELECTED_FOLD:
        copyfile(f'{row.image_path}', f'/kaggle/working/datasets/coco/images/train/{row.image_id}.jpg')
        trainf.write(f'/kaggle/working/datasets/coco/images/train/{row.image_id}.jpg \n')
        with open(f'/kaggle/working/datasets/coco/labels/train/{row.image_id}.txt', "w") as f:
            f.write(row.bboxes)
        
    elif row.fold == SELECTED_FOLD:
        copyfile(f'{row.image_path}', f'/kaggle/working/datasets/coco/images/val/{row.image_id}.jpg')
        trainf.write(f'/kaggle/working/datasets/coco/images/val/{row.image_id}.jpg \n')
        with open(f'/kaggle/working/datasets/coco/labels/val/{row.image_id}.txt', "w") as f:
            f.write(row.bboxes)
trainf.close()
valf.close()

  0%|          | 0/5689 [00:00<?, ?it/s]

In [10]:
%cd /kaggle/working/
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

/kaggle/working
Cloning into 'yolov5'...
remote: Enumerating objects: 10562, done.
remote: Total 10562 (delta 0), reused 0 (delta 0), pack-reused 10562
Receiving objects: 100% (10562/10562), 10.70 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (7304/7304), done.
/kaggle/working/yolov5


In [11]:
import yaml
conf = dict(
train= '/kaggle/working/datasets/coco/images/train/',
val= '/kaggle/working/datasets/coco/images/val/',
nc= 1  ,
names= ['cots']
)
with open('/kaggle/working/yolov5/data/coco.yaml', 'w') as outfile:
    yaml.dump(conf, outfile, default_flow_style=False)

In [12]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 90  # image rotation (+/- deg)
translate: 0.0  # image translation (+/- fraction)
scale: 0.75  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''
with open('/kaggle/working//yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [13]:
!python train.py --img 3000 --batch-size -1 --epochs 20 --data coco.yaml  --project Yolo --name iwant2die  --hyp data/hyps/hyp.heavy.2.yaml --resume /kaggle/input/yolov5nano/yolov5/Yolo/iwant2die/weights/last.pt

wandb: Currently logged in as: othiro (use `wandb login --relogin` to force relogin)
github: up to date with https://github.com/ultralytics/yolov5 ✅
wandb: Tracking run with wandb version 0.12.9
wandb: Resuming run iwant2die
wandb: ⭐️ View project at https://wandb.ai/othiro/Yolo
wandb: 🚀 View run at https://wandb.ai/othiro/Yolo/runs/1bqkoeue
wandb: Run data is saved locally in /kaggle/working/yolov5/wandb/run-20220118_100239-1bqkoeue
wandb: Run `wandb offline` to turn off syncing.
































wandb: Waiting for W&B process to finish, PID 204... (success).
wandb:                                                                                
wandb: Run history:
wandb:        metrics/mAP_0.5 ▅▁▄▆█▃▄▅██
wandb:   metrics/mAP_0.5:0.95 ▄▁▂▄▆▅▅▆██
wandb:      metrics/precision ▅▁▁█▆▃▃▄▆▆
wandb:         metrics/recall ▅▁▄▅█▂▄▅██
wandb:         train/box_loss █▆▅▄▄▃▂▂▁
wandb:         train/cls_loss ▁▁▁▁▁▁▁▁▁
wandb:         train/obj_loss █▅▅▅▅▂▂▁▁
wandb:           val/box_lo